In [2]:
from new_model import * 

In [3]:
seed = 10
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [6]:
## scheduler 학습
def fine_tuning(model, mode, EPOCHS):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed10_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [60, 120, 160], gamma = 0.2)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [7]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

In [8]:
# 모델 지정 및 GPU 지정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 10, 0.3, 100).to(device)
check_prune = torch.load("../1.Wide_ResNet_model/saved_model/WR_28x10_cifar100.pth")
model.load_state_dict(check_prune['state_dict'])
# test_model(model)

| Wide-Resnet 28x10


<All keys matched successfully>

In [9]:
INITIAL_LR = 0.1*0.2

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.2**0.1988

mode = '28x10_0.8_multistepLR_lr0.1*0.2'

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_0.8_CIFAR10_scheduler_multstep_lr0.1*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:0'), tensor(3252., device='cuda:0'))
Epoch 10: progress time is 2696.45 sec
Validation loss: 0.3069, Validation accuracy: 0.9072
Epoch 20: progress time is 5390.52 sec
Validation loss: 0.2786, Validation accuracy: 0.9244
Epoch 30: progress time is 8085.89 sec
Validation loss: 0.3221, Validation accuracy: 0.9076
Epoch 40: progress time is 10779.35 sec
Validation loss: 0.2644, Validation accuracy: 0.9176
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:0'), tensor(2356., device='cuda:0'))
Epoch 50: progress time is 13464.64 sec
Validation loss: 0.2761, Validation accuracy: 0.9176
Epoch 60: progress time is 16152.63 sec
Validation loss: 0.2647, Validation accuracy: 0.9192
Epoch 70: progress time is 18848.83 sec
Validation loss: 0.1451, Validation accuracy: 0.9584
Epoch 80: prog

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x10_0.8_CIFAR10_scheduler_cosine_lr0.1*0.2*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:1'), tensor(3252., device='cuda:1'))
Epoch 10: progress time is 2500.05 sec
Validation loss: 0.1549, Validation accuracy: 0.9564
Epoch 20: progress time is 5029.84 sec
Validation loss: 0.1860, Validation accuracy: 0.9484
Epoch 30: progress time is 7538.61 sec
Validation loss: 0.1612, Validation accuracy: 0.9576
Epoch 40: progress time is 10040.99 sec
Validation loss: 0.1494, Validation accuracy: 0.9600
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:1'), tensor(2356., device='cuda:1'))
Epoch 50: progress time is 12514.56 sec
Validation loss: 0.1780, Validation accuracy: 0.9496
Epoch 60: progress time is 14991.22 sec
Validation loss: 0.1421, Validation accuracy: 0.9580
Epoch 70: progress time is 17472.45 sec
Validation loss: 0.1444, Validation accuracy: 0.9576
Epoch 80: prog

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [11]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:2'), tensor(4480., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:2'), tensor(3252., device='cuda:2'))
Epoch 10: progress time is 2730.83 sec
Validation loss: 0.7905, Validation accuracy: 0.7816
Epoch 20: progress time is 5457.34 sec
Validation loss: 0.8211, Validation accuracy: 0.7708
Epoch 30: progress time is 8189.17 sec
Validation loss: 0.9585, Validation accuracy: 0.7428
Epoch 40: progress time is 10915.06 sec
Validation loss: 1.0700, Validation accuracy: 0.7268
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:2'), tensor(2356., device='cuda:2'))
Epoch 50: progress time is 13629.47 sec
Validation loss: 1.1495, Validation accuracy: 0.7044
Epoch 60: progress time is 16345.65 sec
Validation loss: 1.1268, Validation accuracy: 0.7080
Epoch 70: progress time is 19062.92 sec
Validation loss: 0.7406, Validation accuracy: 0.8024
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [9]:
##scheduler 실행 : cosine, lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:1'), tensor(4480., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:1'), tensor(3252., device='cuda:1'))
Epoch 10: progress time is 2693.79 sec
Validation loss: 0.2550, Validation accuracy: 0.9148
Epoch 20: progress time is 5382.95 sec
Validation loss: 0.2104, Validation accuracy: 0.9380
Epoch 30: progress time is 8071.42 sec
Validation loss: 0.1853, Validation accuracy: 0.9392
Epoch 40: progress time is 10926.26 sec
Validation loss: 0.1569, Validation accuracy: 0.9500
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:1'), tensor(2356., device='cuda:1'))
Epoch 50: progress time is 14243.45 sec
Validation loss: 0.1455, Validation accuracy: 0.9520
Epoch 60: progress time is 18132.52 sec
Validation loss: 0.1555, Validation accuracy: 0.9516
Epoch 70: progress time is 21842.56 sec
Validation loss: 0.1336, Validation accuracy: 0.9616
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## seed 10

In [9]:
##scheduler 실행 : multistepLR, lr = 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:0'), tensor(3252., device='cuda:0'))
Epoch 10: progress time is 2729.96 sec
Validation loss: 0.1302, Validation accuracy: 0.9732
Epoch 20: progress time is 5465.08 sec
Validation loss: 0.1163, Validation accuracy: 0.9812
Epoch 30: progress time is 8197.55 sec
Validation loss: 0.1539, Validation accuracy: 0.9700
Epoch 40: progress time is 10925.39 sec
Validation loss: 0.2671, Validation accuracy: 0.9312
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:0'), tensor(2356., device='cuda:0'))
Epoch 50: progress time is 13674.06 sec
Validation loss: 0.7440, Validation accuracy: 0.7992
Epoch 60: progress time is 16417.33 sec
Validation loss: 0.7233, Validation accuracy: 0.8148
Epoch 70: progress time is 19140.03 sec
Validation loss: 0.5127, Validation accuracy: 0.8692
Epoch 80: progre

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

In [10]:
##scheduler 실행 : multistepLR, lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(4480., device='cuda:0'), tensor(4480., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(3252., device='cuda:0'), tensor(3252., device='cuda:0'))
Epoch 10: progress time is 2742.5 sec
Validation loss: 0.8901, Validation accuracy: 0.7612
Epoch 20: progress time is 5486.55 sec
Validation loss: 0.8760, Validation accuracy: 0.7648
Epoch 30: progress time is 8220.39 sec
Validation loss: 0.9948, Validation accuracy: 0.7412
Epoch 40: progress time is 10951.8 sec
Validation loss: 1.0115, Validation accuracy: 0.7336
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(2356., device='cuda:0'), tensor(2356., device='cuda:0'))
Epoch 50: progress time is 13679.76 sec
Validation loss: 1.2967, Validation accuracy: 0.6732
Epoch 60: progress time is 16407.41 sec
Validation loss: 1.2026, Validation accuracy: 0.6996
Epoch 70: progress time is 19138.97 sec
Validation loss: 0.7545, Validation accuracy: 0.8100
Epoch 80: progress

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(160, 160, kernel_si

## 결과 확인

In [10]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.8_CIFAR10_scheduler_multstep_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x10_0.8_CIFAR10_scheduler_multstep finished')

Total examples is 10000, correct examples is 9523; Test accuracy: 0.9523


In [10]:
## cifar10
check_prune = torch.load("./saved_tuning/28x10_0.8_CIFAR10_scheduler_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x10_0.8_CIFAR10_scheduler_cosinefinished')

Total examples is 10000, correct examples is 9545; Test accuracy: 0.9545


In [12]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x10_0.8_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7726; Test accuracy: 0.7726


## seed 10

In [ ]:
## cifar100
check_prune = torch.load("./saved_seed_CIFAR10/28x10_0.8_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x10_0.6_multistepLR_lr0.1*0.2*0.2 finished')

In [15]:
os.getcwd()

'/home/yeonho/final_saved_model/3.node_sparse(fine tuning)'

In [16]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.8_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

Total examples is 10000, correct examples is 9535; Test accuracy: 0.9535


In [10]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x10_0.8_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

Total examples is 10000, correct examples is 9551; Test accuracy: 0.9551


In [11]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.8_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

Total examples is 10000, correct examples is 7821; Test accuracy: 0.7821


In [11]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x10_0.8_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

Total examples is 10000, correct examples is 7802; Test accuracy: 0.7802
